In [ ]:
import pandas as pd
import pickle, os, json
import scipy.io as sio
from bci_funcs import slope
import numpy as np

path = '/Users/lukasgehrke/data/fastReach/data/eeglab2python'

pIDs = [12, 14, 15, 16, 17, 19, 20, 21, 22, 23]

for pID in pIDs:
    pID = 'sub-' + "%03d" % (pID)

    # load design matrix
    design = pd.read_csv(path+os.sep+pID+os.sep+'design.csv')
    # load mat bad noisy_trials_pre_move
    noisy_trials = sio.loadmat(path+os.sep+pID+os.sep+'noisy_trials_pre_move.mat')['noisy_trials_pre_move']
    # remove noisy trials from design
    design = design.drop(noisy_trials.flatten(), axis=0).reset_index(drop=True)

    # load an sav file
    filename = 'model_sub-023_eeg.sav'
    loaded_model = pickle.load(open(path+os.sep+pID+os.sep+'model_'+pID+'_eeg.sav', 'rb'))
    # load bci_params.json
    with open(path+os.sep+pID+os.sep+'bci_params.json') as f:
        bci_params = json.load(f)

    # load data
    data = sio.loadmat(path+os.sep+pID+os.sep+'pre_move_processed_-1_1.mat')['pre_move']
    fcz = data[-1,:,:]
    data = data[bci_params['chans'],:,:]

    # compute feature
    feat = np.zeros((data.shape[2], data.shape[0]))
    for trial_ix in design.index:

        ems_onset = design['delta_tap_ems'][trial_ix]

        if np.isnan(ems_onset):
            ems_onset = 500
        else:
            ems_onset = 500 - int(ems_onset * 250)

            if ems_onset < 250:
                ems_onset = 250

        feat[trial_ix, :] = slope(data[:,ems_onset-250:ems_onset,trial_ix], 'linear').flatten()

    proba = loaded_model.predict_proba(feat)

    # fcz_n1 = fcz[524,:]
    n1 = data[:,524,:]

    # correlate n1[0,:] with proba
    # from scipy.stats import pearsonr
    # print(pearsonr(n1[-1,:], proba[:,1]))

    # add proba to design
    design['proba'] = proba[:,1]

In [ ]:
design

In [ ]:
# Regression model on single time points of ERP
# erp ~ time_estimation + tap_source (self/ems) + lda score / probability

n1 = data[:,524,:]
fcz_n1 = fcz[524,:]

# correlate n1[0,:] with proba
from scipy.stats import pearsonr
pearsonr(fcz_n1, proba[:,1])


In [ ]:
# select condition 'baseline' from design
baseline = design[design['condition'] == 'baseline']
baseline_fcz = fcz[:,baseline.index]
# baseline correct with index 449 to 500
baseline_fcz = baseline_fcz - baseline_fcz[449:500,:].mean(axis=0)

ems = design[design['condition'] == 'ems_bci']
ems_fcz = fcz[:,ems.index]
# baseline correct with index 449 to 500
ems_fcz = ems_fcz - ems_fcz[449:500,:].mean(axis=0)

external = design[design['condition'] == 'ems_random']
external_fcz = fcz[:,external.index]
# baseline correct with index 449 to 500
external_fcz = external_fcz - external_fcz[449:500,:].mean(axis=0)

# plot fcz mean 2nd dim
import matplotlib.pyplot as plt
# plt.plot(baseline_fcz.mean(axis=1))
plt.plot(external_fcz.mean(axis=1) - ems_fcz.mean(axis=1))
# plt.plot(external_fcz.mean(axis=1))
# xline at 500
plt.axvline(500, color='k', linestyle='--')

In [ ]:
# select control 'muscle' from design
muscle = design[design['control'] == 'muscle']
muscle_fcz = fcz[:,muscle.index]
# baseline correct with index 449 to 500
muscle_fcz = muscle_fcz - muscle_fcz[449:500,:].mean(axis=0)

# ems from control
muscle_ems = design[design['control'] == 'ems']
muscle_ems_fcz = fcz[:,muscle_ems.index]
# baseline correct with index 449 to 500
muscle_ems_fcz = muscle_ems_fcz - muscle_ems_fcz[449:500,:].mean(axis=0)

# plot fcz mean 2nd dim
import matplotlib.pyplot as plt
plt.plot(muscle_fcz.mean(axis=1))
plt.plot(muscle_ems_fcz.mean(axis=1))